# A preliminary introduction to [PyGMT](https://www.pygmt.org/)

- **Presenter**: [YAO Jiayuan](https://github.com/core-man)
- **Date**: 16 March 2021

---
## 1. Making your first figure

First, let's import the PyGMT Python package:

In [ ]:
import pygmt

All figure generation in PyGMT is handled by the [`pygmt.Figure`](https://www.pygmt.org/latest/api/generated/pygmt.Figure.html#pygmt.Figure) class. Every figure must start with the creation of an instance of this class:

In [ ]:
fig = pygmt.Figure()

Add elements to the figure using its methods. For example, let’s use [`pygmt.Figure.basemap`](https://www.pygmt.org/latest/api/generated/pygmt.Figure.basemap.html#pygmt.Figure.basemap) to start the creation of a map. We’ll use the `region` parameter to provide the longitude and latitude bounds, the `projection` parameter to set the projection to Mercator (**M**) and the map width to 15 cm, and the `frame` parameter to generate a frame with automatic tick and annotation spacings.

In [ ]:
fig.basemap(region=[-90, -70, 0, 20], projection="M15c", frame=True)

Now we can add coastlines using [`pygmt.Figure.coast`](https://www.pygmt.org/latest/api/generated/pygmt.Figure.coast.html#pygmt.Figure.coast) to this map using the default resolution, line width, and color:

In [ ]:
fig.coast(shorelines=True)

To see the figure, call [`pygmt.Figure.show`](https://www.pygmt.org/latest/api/generated/pygmt.Figure.show.html#pygmt.Figure.show):

In [ ]:
fig.show()

We can also set the map region, projection, and frame type directly in other methods without calling [`gmt.Figure.basemap`](https://www.pygmt.org/latest/api/generated/pygmt.Figure.basemap.html#pygmt.Figure.basemap):

In [ ]:
fig = pygmt.Figure()
fig.coast(region=[-90, -70, 0, 20], projection="M15c", shorelines=True, frame=True)
fig.show()

We can also show a figure "externally" in a PDF viewer (e.g., in Preview on macOS) with the `method='external'` keyword argument:

In [ ]:
fig.show(method='external')

Use the method [`pygmt.Figure.savefig`](https://www.pygmt.org/latest/api/generated/pygmt.Figure.savefig.html#pygmt.Figure.savefig) to save your figure to a file. The figure format is inferred from the extension.

In [ ]:
fig.savefig("central-america-shorelines.png")

## 2. Notes for experienced GMT users

You have probably noticed several things that are different from classic command-line GMT. Many of these changes reflect the new GMT modern execution mode that is part of GMT 6.
1. As a general rule, the `ps` prefix has been removed from all `ps*` modules (PyGMT methods). For example, the name of the GMT 5 module `pscoast` is `coast` in GMT 6 and PyGMT.
  The exceptions are: `psxy` which is now `plot`, `psxyz` which is now `plot3d`, and `psscale` which is now `colorbar`.
2. More details can be found in the [GMT cookbook introduction to modern mode](https://docs.generic-mapping-tools.org/latest//cookbook/introduction.html#modern-and-classic-mode).

A few are PyGMT exclusive (like the `savefig` method).

1. The PyGMT parameters (called options or arguments in GMT) don’t use the GMT 1-letter syntax (**R**, **J**, **B**, etc). We use longer aliases for these parameters and have some Python exclusive names. The mapping between the GMT parameters and their PyGMT aliases should be straightforward. For some modules, these aliases are still being developed. For example, `region` is the alias for **R** and `projection` is for **J**, and `frame` is for **B**.
2. Parameters like `region` can take **lists** as well as **strings** like `1/2/3/4`.
3. If a GMT option has no arguments (like `-B` instead of `-Baf`), use a `True` in Python. An empty string would also be acceptable. For repeated parameters, such as `-B+Loleron -Bxaf -By+lm`, provide a **list**: `frame=["+Loleron", "xaf", "y+lm"]`.
4. There is no output redirecting to a PostScript file. The figure is generated in the background and will only be shown or saved when you ask for it.

## 3. Some gallery examples

Let's see a few more examples from PyGMT [tutorial](https://www.pygmt.org/latest/tutorials/) and [gallery](https://www.pygmt.org/latest/gallery/index.html).

In [ ]:
import numpy as np

**Plot basemap:** [Double Y axes graph](https://www.pygmt.org/latest/gallery/basemaps/double_y_axes.html#sphx-glr-gallery-basemaps-double-y-axes-py)

In [ ]:
# Generate two sample Y data from one common X data
x = np.linspace(1.0, 9.0, num=9)
y1 = x
y2 = x ** 2 + 110

fig = pygmt.Figure()

# Plot the common X axes
# The bottom axis (S) is plotted with annotations and tick marks
# The top axis (t) is plotted without annotations and tick marks
# The left and right axes are not drawn
fig.basemap(region=[0, 10, 0, 10], projection="X15c/15c", frame=["St", "xaf+lx"])

# Plot the Y axis for y1 data
# The left axis (W) is plotted with blue annotations, ticks, and label
with pygmt.config(
    MAP_FRAME_PEN="blue",
    MAP_TICK_PEN="blue",
    FONT_ANNOT_PRIMARY="blue",
    FONT_LABEL="blue",
):
    fig.basemap(frame=["W", "yaf+ly1"])

# Plot the line for y1 data
fig.plot(x=x, y=y1, pen="1p,blue")
# Plot points for y1 data
fig.plot(x=x, y=y1, style="c0.2c", color="blue", label="y1")

# Plot the Y axis for y2 data
# The right axis (E) is plotted with red annotations, ticks, and label
with pygmt.config(
    MAP_FRAME_PEN="red",
    MAP_TICK_PEN="red",
    FONT_ANNOT_PRIMARY="red",
    FONT_LABEL="red",
):
    fig.basemap(region=[0, 10, 100, 200], frame=["E", "yaf+ly2"])
# Plot the line for y2 data
fig.plot(x=x, y=y2, pen="1p,red")
# Plot points for y2 data
fig.plot(x=x, y=y2, style="s0.28c", color="red", label="y2")

# Create a legend in the top-left corner of the plot
fig.legend(position="jTL+o0.1c", box=True)

fig.show()

**Plotting Earth relief:** [Plot a region map](https://www.pygmt.org/latest/tutorials/earth_relief.html#create-a-region-map)

In [ ]:
grid = pygmt.datasets.load_earth_relief(resolution="10m", region=[-14, 30, 35, 60])

fig = pygmt.Figure()
fig.grdimage(grid=grid, projection="M15c", frame="a", cmap="geo")
fig.colorbar(frame=["a1000", "x+lElevation", "y+lm"])
fig.show()

**Lines and vectors:** [Vector heads and tails](https://www.pygmt.org/latest/gallery/lines/vector_heads_tails.html#sphx-glr-gallery-lines-vector-heads-tails-py)

In [ ]:
fig = pygmt.Figure()
fig.basemap(
    region=[0, 10, 0, 15], projection="X15c/10c", frame='+t"Vector heads and tails"'
)

x = 1
y = 14
angle = 0  # in degrees, measured counter-clockwise from horizontal
length = 7

for vecstyle in [
    # vector without head and tail (line)
    "v0c",
    # plain open arrow at beginning and end, angle of the vector head apex is set to 50
    "v0.6c+bA+eA+a50",
    # plain open tail at beginning and end
    "v0.4c+bI+eI",
    # terminal line at beginning and end, angle of vector head apex is set to 80
    "v0.3c+bt+et+a80",
    # arrow head at end
    "v0.6c+e",
    # circle at beginning and arrow head at end
    "v0.6c+bc+ea",
    # terminal line at beginning and arrow head at end
    "v0.6c+bt+ea",
    # arrow head at end, shape of vector head is set to 0.5
    "v1c+e+h0.5",
    # modified arrow heads at beginning and end
    "v1c+b+e+h0.5",
    # tail at beginning and arrow with modified vector head at end
    "v1c+bi+ea+h0.5",
    # half-sided arrow head (right side) at beginning and arrow at the end
    "v1c+bar+ea+h0.8",
    # half-sided arrow heads at beginning (right side) and end (left side)
    "v1c+bar+eal+h0.5",
    # half-sided tail at beginning and arrow at end (right side for both)
    "v1c+bi+ea+r+h0.5+a45",
]:
    fig.plot(
        x=x, y=y, style=vecstyle, direction=([angle], [length]), pen="2p", color="red3"
    )
    fig.text(
        x=6, y=y, text=vecstyle, font="Courier-Bold", justify="ML", offset="0.2c/0c"
    )
    y -= 1  # move the next vector down

fig.show()

**Symbols:** [Plotting data points](https://www.pygmt.org/latest/tutorials/plot.html#)

In [ ]:
data = pygmt.datasets.load_japan_quakes()

# Set the region for the plot to be slightly larger than the data bounds.
region = [
    data.longitude.min() - 1,
    data.longitude.max() + 1,
    data.latitude.min() - 1,
    data.latitude.max() + 1,
]

print(region)
print(data.head())

fig = pygmt.Figure()

fig.basemap(region=region, projection="M15c", frame=True)
fig.coast(land="black", water="skyblue")

pygmt.makecpt(cmap="viridis", series=[data.depth_km.min(), data.depth_km.max()])
fig.plot(
    x=data.longitude,
    y=data.latitude,
    style="cc",
    sizes=0.02 * 2 ** data.magnitude,
    color=data.depth_km,
    cmap=True,
    pen="black",
)
fig.colorbar(frame='af+l"Depth (km)"')

fig.show()

**Plot embellishments:** [Inset map showing a rectangular region](https://www.pygmt.org/latest/gallery/embellishments/inset_rectangle_region.html#sphx-glr-gallery-embellishments-inset-rectangle-region-py)

In [ ]:
# Set the region of the main figure
region = [137.5, 141, 34, 37]

fig = pygmt.Figure()

# Plot the base map of the main figure. Universal Transverse Mercator (UTM) projection
# is used and the UTM zone is set to be "54S".
fig.basemap(region=region, projection="U54S/12c", frame=["WSne", "af"])

# Set the land color to "lightbrown", the water color to "azure1", the shoreline
# width to "2p", and the area threshold to 1000 km^2 for the main figure
fig.coast(land="lightbrown", water="azure1", shorelines="2p", area_thresh=1000)

# Create an inset map, setting the position to bottom right, the width to
# 3 cm, the height to 3.6 cm, and the x- and y-offsets to
# 0.1 cm, respectively. Draws a rectangular box around the inset with a fill color
# of "white" and a pen of "1p".
with fig.inset(position="jBR+w3c/3.6c+o0.1c", box="+gwhite+p1p"):
    # Plot the Japan main land in the inset using coast. "U54S/M?" means UTM
    # projection with map width automatically determined from the inset width.
    # Highlight the Japan area in "lightbrown"
    # and draw its outline with a pen of "0.2p".
    fig.coast(
        region=[129, 146, 30, 46],
        projection="U54S/?",
        dcw="JP+glightbrown+p0.2p",
        area_thresh=10000,
    )
    # Plot a rectangle ("r") in the inset map to show the area of the main figure.
    # "+s" means that the first two columns are the longitude and latitude of
    # the bottom left corner of the rectangle, and the last two columns the
    # longitude and latitude of the uppper right corner.
    rectangle = [[region[0], region[2], region[1], region[3]]]
    fig.plot(data=rectangle, style="r+s", pen="2p,blue")

fig.show()

## 4. Accessing GMT commands using PyGMT

PyGMT works by: 1) wrapping GMT commands (called "modules" in GMT lingo) in a more "Pythonic" shell; and 2) facilitating Python data objects to be passed to GMT commands. GMT commands are getting added regularly — but if you don't find the one you need in the API reference, you can still call any GMT command using PyGMT using the [`pygmt.clib.Session`](https://www.pygmt.org/latest/api/generated/pygmt.clib.Session.html#pygmt.clib.Session) class. Below, an example using GMT's [`meca`](https://docs.generic-mapping-tools.org/latest/supplements/seis/meca.html) command, which plots focal mechanisms:

In [ ]:
fig = pygmt.Figure()

# Generate a basemap near Washington state showing coastlines, land, and water
fig.coast(
    region=[-125, -122, 47, 49],
    projection="M6c",
    land="grey",
    water="lightblue",
    shorelines=True,
    frame="a",
)

# Use a few context managers to call meca
with pygmt.helpers.GMTTempFile() as temp_file:
    with open(temp_file.name, 'w') as f:
        f.write('-124.3 48.1 12.0 330 30 90 3 0 0')
    with pygmt.clib.Session() as session:
        session.call_module('meca', f'{temp_file.name} -Sa1c')

fig.show()

In [ ]:
fig = pygmt.Figure()

# Generate a basemap near Washington state showing coastlines, land, and water
fig.coast(
    region=[-125, -122, 47, 49],
    projection="M6c",
    land="grey",
    water="lightblue",
    shorelines=True,
    frame="a",
)

# Use a few context managers to call meca
with pygmt.helpers.GMTTempFile() as temp_file:
    with open(temp_file.name, 'w') as f:
        f.write('-124.3 48.1 12.0 330 30 90 3 0 0')
    with pygmt.clib.Session() as session:
        session.call_module('meca', f'{temp_file.name} -Sa1c')


# Store focal mechanisms parameters in a dict
focal_mechanism = dict(strike=330, dip=30, rake=90, magnitude=3)

# Pass the focal mechanism data to meca in addition to the scale and event location
fig.meca(focal_mechanism, longitude=-123.3, latitude=48.1, depth=12.0, scale="1c")

fig.show()

## 5. Resources

* Documentation: [pygmt.org](https://www.pygmt.org/latest/)
* Documentation (development version = master branch on GitHub): [pygmt.org/dev](https://www.pygmt.org/dev/)
* If documentation doesn't have the answer, ask on the GMT forum under the Q&A category: [forum.generic-mapping-tools.org](https://forum.generic-mapping-tools.org/)
* If you think something isn't working right, file an Issue on GitHub: [github.com/GenericMappingTools/pygmt](https://github.com/GenericMappingTools/pygmt)

All of PyGMT's functionality is described at the above documentation website. It's your one-stop shop for both detailed documentation **and** helpful examples and tutorials. The [API Reference](https://www.pygmt.org/latest/api/index.html) section (API = Application Programming Interface) shows all the class and methods in PyGMT. Let's look at the documentation.